# 🧠 MS Lesion Synthesis Inference Notebook (biomarkem data)

In [1]:
import os
from pathlib import Path
import ants
import numpy as np
from PIL import Image

### Extract the GM/WM masks from all the biomarkem data

In [3]:
# Define base path and scanners
base_path = Path("/home/benet/data/biomarkem2D")
scanners = ["ISI", "TSI", "VSI"]


# Define output path for WM/GM masks
output_base = Path("./biomarkem/WM_GM_MASKS")
output_base.mkdir(parents=True, exist_ok=True)

# Collect image paths and prepare segmentation
segmented_images = []

for scanner in scanners:
    print(f"Processing scanner: {scanner}")
    scanner_path = base_path / scanner
    output_path = output_base / scanner
    output_path.mkdir(parents=True, exist_ok=True)

    for fname in os.listdir(scanner_path):
        if not fname.endswith(".png"):
            continue

        img_path = scanner_path / fname
        img_arr = np.array(Image.open(img_path).convert("L")).astype(np.float32)

        # Segment using ANTs
        img = ants.from_numpy(img_arr)
        mask = ants.get_mask(img)
        seg = ants.atropos(
            a=img,
            m='[0.2,1x1]',
            c='[2,0]',
            i='kmeans[3]',
            x=mask,
            verbose=0
        )

        # Classify tissues by intensity
        seg_img = seg["segmentation"]
        labels = np.unique(seg_img.numpy())
        means = [img.numpy()[seg_img.numpy() == l].mean() for l in labels]
        sorted_labels = [label for _, label in sorted(zip(means, labels))]

        # Combine WM and GM (highest two intensity classes)
        wm_gm_mask = np.isin(seg_img.numpy(), sorted_labels[-2:]).astype(np.uint8) * 255

        # Save mask
        output_file = output_path / fname
        Image.fromarray(wm_gm_mask).save(output_file)
        segmented_images.append(str(output_file))

Processing scanner: ISI
Processing scanner: TSI
Processing scanner: VSI


### Create new lesion masks from lesions masks in the test data and that are in the GM/WM masks

In [ ]:
# def generate_synthetic_mask_for_folder(lesion_dict, wm_mask_folder, output_folder,
#                                        num_lesions={'small': 1, 'medium': 1, 'large': 1},
#                                        seed=17844):
#     wm_mask_folder = Path(wm_mask_folder)
#     output_folder = Path(output_folder)
#     output_folder.mkdir(parents=True, exist_ok=True)

#     for fname in os.listdir(wm_mask_folder):
#         if not fname.endswith(".png"):
#             continue

#         wm_mask_path = wm_mask_folder / fname
#         wm_mask = np.array(Image.open(wm_mask_path).convert("L"))
#         shape = wm_mask.shape

#         syn_mask = paste_lesions(shape, lesion_dict, wm_mask, num_lesions, seed=seed)

#         output_path = output_folder / fname
#         Image.fromarray(syn_mask).save(output_path)

# def batch_generate_synthetic_masks(source_mask_dir, wm_base_folder, output_base_folder,
#                                    scanners=["ISI", "TSI", "VSI"],
#                                    num_lesions={'small': 1, 'medium': 1, 'large': 1},
#                                    seed=17844):
#     print("Loading and categorizing lesion bank...")
#     lesion_dict = load_and_categorize_lesions(source_mask_dir)

#     print("Generating synthetic masks per scanner...")
#     for scanner in scanners:
#         wm_mask_folder = os.path.join(wm_base_folder, scanner)
#         output_folder = os.path.join(output_base_folder, scanner)
#         print(f"Processing {scanner}...")
#         generate_synthetic_mask_for_folder(lesion_dict, wm_mask_folder, output_folder,
#                                            num_lesions=num_lesions, seed=seed)

#     print("✅ All synthetic masks generated.")

# batch_generate_synthetic_masks(
#     source_mask_dir="/home/benet/data/lesion2D_VH-SHIFTS-WMH2017/test/mask",
#     wm_base_folder="./biomarkem/WM_GM_MASKS",
#     output_base_folder="./biomarkem/synthetic_masks",
#     num_lesions={'small': 0, 'medium': 0, 'large': 3},
#     seed=17844
# )


Loading and categorizing lesion bank...
Generating synthetic masks per scanner...
Processing ISI...
Processing TSI...
Processing VSI...
✅ All synthetic masks generated.


In [ ]:
import os
import random
import numpy as np
import cv2
from scipy.ndimage import label
from pathlib import Path
from PIL import Image

def extract_lesions(mask):
    labeled, n = label(mask)
    lesions = []
    for i in range(1, n + 1):
        lesion = (labeled == i).astype(np.uint8) * 255
        area = np.sum(lesion > 0)
        lesions.append((lesion, area))
    return lesions

def categorize_lesions(lesions):
    areas = [a for _, a in lesions]
    if len(areas) < 3:
        return {'small': [], 'medium': [], 'large': []}
    small_th = np.percentile(areas, 33)
    large_th = np.percentile(areas, 80)
    categorized = {'small': [], 'medium': [], 'large': []}
    for lesion, area in lesions:
        if area <= small_th:
            categorized['small'].append(lesion)
        elif area <= large_th:
            categorized['medium'].append(lesion)
        else:
            categorized['large'].append(lesion)
    return categorized

def load_and_categorize_lesions(source_mask_dir):
    lesion_bank = []
    for fname in os.listdir(source_mask_dir):
        if fname.endswith(".png"):
            mask = np.array(Image.open(os.path.join(source_mask_dir, fname)).convert("L"))
            lesion_bank.extend(extract_lesions(mask))
    return categorize_lesions(lesion_bank)

def transform_lesion(lesion, target_shape):
    h, w = lesion.shape
    scale = random.uniform(0.8, 1.2)
    angle = random.uniform(-20, 20)
    lesion_resized = cv2.resize(lesion, None, fx=scale, fy=scale)
    M = cv2.getRotationMatrix2D((lesion_resized.shape[1] // 2, lesion_resized.shape[0] // 2), angle, 1)
    lesion_rotated = cv2.warpAffine(lesion_resized, M, (lesion_resized.shape[1], lesion_resized.shape[0]))
    lh, lw = lesion_rotated.shape
    if lh >= target_shape[0] or lw >= target_shape[1]:
        return None
    return lesion_rotated

def paste_lesions(target_shape, lesion_dict, wm_mask, num_lesions, seed=None):
    if seed is not None:
        random.seed(seed)
        np.random.seed(seed)

    new_mask = np.zeros(target_shape, dtype=np.uint8)

    for cat in ['small', 'medium', 'large']:
        count = num_lesions.get(cat, 0)
        if not lesion_dict[cat]:
            print(f"No lesions available for category {cat}")
            continue

        placed = 0
        tries = 0
        while placed < count and tries < 100:
            lesion = random.choice(lesion_dict[cat])
            lesion_transformed = transform_lesion(lesion, target_shape)
            if lesion_transformed is None:
                tries += 1
                continue

            lh, lw = lesion_transformed.shape
            x_offset = random.randint(0, target_shape[0] - lh)
            y_offset = random.randint(0, target_shape[1] - lw)

            lesion_bin = (lesion_transformed > 0).astype(np.uint8)
            region = wm_mask[x_offset:x_offset+lh, y_offset:y_offset+lw]
            region_bin = (region > 0).astype(np.uint8)

            if not np.all(region_bin[lesion_bin == 1] == 1):
                tries += 1
                continue

            new_mask[x_offset:x_offset+lh, y_offset:y_offset+lw] = np.maximum(
                new_mask[x_offset:x_offset+lh, y_offset:y_offset+lw], lesion_bin * 255)
            placed += 1

    return (new_mask > 0).astype(np.uint8) * 255

In [26]:
def generate_synthetic_mask_for_suffixes(lesion_dict, wm_base_folder, output_base_folder,
                                         scanners=["ISI", "TSI", "VSI"],
                                         num_lesions={'small': 1, 'medium': 1, 'large': 1},
                                         seed=17844):
    wm_base_folder = Path(wm_base_folder)
    output_base_folder = Path(output_base_folder)
    output_base_folder.mkdir(parents=True, exist_ok=True)

    # Collect common suffixes
    suffixes = set()
    for scanner in scanners:
        for fname in os.listdir(wm_base_folder / scanner):
            if fname.endswith(".png") and fname.startswith(scanner):
                suffixes.add(fname[len(scanner):])  # strip scanner prefix

    for suffix in sorted(suffixes):
        print(f"Generating synthetic masks for suffix: {suffix}")
        wm_masks = []
        shape = None
        for scanner in scanners:
            path = wm_base_folder / scanner / f"{scanner}{suffix}"
            if not path.exists():
                print(f"Missing: {path}")
                break
            wm = np.array(Image.open(path).convert("L"))
            wm_masks.append(wm)
            shape = wm.shape
        else:
            # Try generating a mask that fits in all WM masks
            max_tries = 100
            for _ in range(max_tries):
                syn_mask = paste_lesions(shape, lesion_dict, wm_masks[0], num_lesions, seed)
                if all(np.all((syn_mask > 0) <= (wm > 0)) for wm in wm_masks[1:]):
                    for scanner in scanners:
                        out_path = output_base_folder / scanner
                        out_path.mkdir(parents=True, exist_ok=True)
                        Image.fromarray(syn_mask).save(out_path / f"{scanner}{suffix}")
                    break
            else:
                print(f"❌ Could not generate valid mask for suffix {suffix}")

def batch_generate_unified_synthetic_masks(source_mask_dir, wm_base_folder, output_base_folder,
                                           scanners=["ISI", "TSI", "VSI"],
                                           num_lesions={'small': 1, 'medium': 1, 'large': 1},
                                           seed=17844):
    print("Loading and categorizing lesion bank...")
    lesion_dict = load_and_categorize_lesions(source_mask_dir)

    print("Generating unified synthetic masks per suffix...")
    generate_synthetic_mask_for_suffixes(lesion_dict, wm_base_folder, output_base_folder,
                                         scanners=scanners, num_lesions=num_lesions, seed=seed)
    print("✅ Unified synthetic masks generated.")


In [30]:
batch_generate_unified_synthetic_masks(
    source_mask_dir="/home/benet/data/lesion2D_VH-SHIFTS-WMH2017/test/mask",
    wm_base_folder="./biomarkem/WM_GM_MASKS",
    output_base_folder="./biomarkem/synthetic_masks_unified",
    num_lesions={'small': 0, 'medium': 0, 'large': 3},
    seed=17844
)


Loading and categorizing lesion bank...
Generating unified synthetic masks per suffix...
Generating synthetic masks for suffix: 01_0_0.png
❌ Could not generate valid mask for suffix 01_0_0.png
Generating synthetic masks for suffix: 01_0_1.png
Generating synthetic masks for suffix: 01_0_2.png
❌ Could not generate valid mask for suffix 01_0_2.png
Generating synthetic masks for suffix: 01_0_3.png
❌ Could not generate valid mask for suffix 01_0_3.png
Generating synthetic masks for suffix: 01_0_4.png
❌ Could not generate valid mask for suffix 01_0_4.png
Generating synthetic masks for suffix: 02_0_0.png
❌ Could not generate valid mask for suffix 02_0_0.png
Generating synthetic masks for suffix: 02_0_1.png
❌ Could not generate valid mask for suffix 02_0_1.png
Generating synthetic masks for suffix: 02_0_2.png
Generating synthetic masks for suffix: 02_0_3.png
❌ Could not generate valid mask for suffix 02_0_3.png
Generating synthetic masks for suffix: 02_0_4.png
❌ Could not generate valid mask f

### Generate lesions

In [42]:
from diffusers import StableDiffusionInpaintPipeline
from torchvision import transforms
from PIL import Image
import torch
import os
from pathlib import Path
from tqdm import tqdm

def load_image(path, size=512):
    image = Image.open(path).convert("RGB")
    image = transforms.Compose([
        transforms.Resize(size),
        transforms.CenterCrop(size),
    ])(image)
    return image

def generate_guidance_variants_by_scanner(
    model_id,
    flair_root="/home/benet/data/biomarkem2D",
    mask_root="./biomarkem/synthetic_masks",
    output_root="./biomarkem/generated_inpaintings",
    guidance_values=[0.0, 1.0, 2.0, 3.0, 5.0, 7.5, 10.0],
    image_size=512,
    seed=17844,
    device="cuda"
):
    # Load the inpainting model
    pipe = StableDiffusionInpaintPipeline.from_pretrained(model_id, torch_dtype=torch.float16)
    pipe.to(device)
    pipe.set_progress_bar_config(disable=True)

    generator = torch.manual_seed(seed)
    flair_root = Path(flair_root)
    mask_root = Path(mask_root)
    output_root = Path(output_root)
    output_root.mkdir(parents=True, exist_ok=True)

    for scanner in ["VSI", "TSI", "ISI"]:
        flair_folder = flair_root / scanner
        mask_folder = mask_root / scanner
        output_scanner = output_root / scanner
        output_scanner.mkdir(parents=True, exist_ok=True)

        print(f"🧠 Processing scanner: {scanner}")
        for fname in tqdm(sorted(os.listdir(flair_folder)), desc=f"{scanner}"):
            if not fname.endswith(".png"):
                continue

            flair_path = flair_folder / fname
            mask_path = mask_folder / fname
            if not mask_path.exists():
                print(f"⚠️  Skipping {fname}: mask not found at {mask_path}")
                continue

            image = load_image(flair_path, image_size)
            mask = load_image(mask_path, image_size)

            image_out_dir = output_scanner / fname.replace(".png", "")
            image_out_dir.mkdir(parents=True, exist_ok=True)

            for g in guidance_values:
                result = pipe(
                    prompt="SHIFTS multiple sclerosis lesion in a FLAIR MRI",
                    image=image,
                    mask_image=mask,
                    guidance_scale=g,
                    num_inference_steps=100, #25,
                    generator=generator
                ).images[0]

                out_path = image_out_dir / f"g{g}.png"
                result.save(out_path)

    print("✅ All guidance inpaintings generated.")

In [43]:
generate_guidance_variants_by_scanner(
    model_id="benetraco/ms-lesion-inpainting-vh-shifts-wmh2017_v2",
    flair_root="/home/benet/data/biomarkem2D",
    mask_root="./biomarkem/synthetic_masks_unified",
    output_root="./biomarkem/generated_inpaintings_unified_100",
    guidance_values=[0.0, 1.0, 2.0, 3.0, 5.0, 7.5, 10.0],
    image_size=512,
    seed=17844,
    device="cuda"
)


Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

You have disabled the safety checker for <class 'diffusers.pipelines.stable_diffusion.pipeline_stable_diffusion_inpaint.StableDiffusionInpaintPipeline'> by passing `safety_checker=None`. Ensure that you abide to the conditions of the Stable Diffusion license and do not expose unfiltered results in services or applications open to the public. Both the diffusers team and Hugging Face strongly recommend to keep the safety filter enabled in all public facing circumstances, disabling it only for use-cases that involve analyzing network behavior or auditing its results. For more information, please have a look at https://github.com/huggingface/diffusers/pull/254 .


🧠 Processing scanner: VSI


VSI:   0%|          | 0/75 [00:00<?, ?it/s]

⚠️  Skipping VSI01_0_0.png: mask not found at biomarkem/synthetic_masks_unified/VSI/VSI01_0_0.png


...

⚠️  Skipping ISI11_0_1.png: mask not found at biomarkem/synthetic_masks_unified/ISI/ISI11_0_1.png


ISI:  71%|███████   | 53/75 [08:58<04:05, 11.15s/it]

⚠️  Skipping ISI11_0_3.png: mask not found at biomarkem/synthetic_masks_unified/ISI/ISI11_0_3.png
⚠️  Skipping ISI11_0_4.png: mask not found at biomarkem/synthetic_masks_unified/ISI/ISI11_0_4.png
⚠️  Skipping ISI12_0_0.png: mask not found at biomarkem/synthetic_masks_unified/ISI/ISI12_0_0.png
⚠️  Skipping ISI12_0_1.png: mask not found at biomarkem/synthetic_masks_unified/ISI/ISI12_0_1.png
⚠️  Skipping ISI12_0_2.png: mask not found at biomarkem/synthetic_masks_unified/ISI/ISI12_0_2.png
⚠️  Skipping ISI12_0_3.png: mask not found at biomarkem/synthetic_masks_unified/ISI/ISI12_0_3.png
⚠️  Skipping ISI12_0_4.png: mask not found at biomarkem/synthetic_masks_unified/ISI/ISI12_0_4.png
⚠️  Skipping ISI13_0_0.png: mask not found at biomarkem/synthetic_masks_unified/ISI/ISI13_0_0.png
⚠️  Skipping ISI13_0_1.png: mask not found at biomarkem/synthetic_masks_unified/ISI/ISI13_0_1.png
⚠️  Skipping ISI13_0_2.png: mask not found at biomarkem/synthetic_masks_unified/ISI/ISI13_0_2.png
⚠️  Skipping ISI13_0

ISI: 100%|██████████| 75/75 [10:13<00:00,  8.18s/it]

⚠️  Skipping ISI14_0_3.png: mask not found at biomarkem/synthetic_masks_unified/ISI/ISI14_0_3.png
⚠️  Skipping ISI14_0_4.png: mask not found at biomarkem/synthetic_masks_unified/ISI/ISI14_0_4.png
⚠️  Skipping ISI15_0_0.png: mask not found at biomarkem/synthetic_masks_unified/ISI/ISI15_0_0.png
⚠️  Skipping ISI15_0_1.png: mask not found at biomarkem/synthetic_masks_unified/ISI/ISI15_0_1.png
⚠️  Skipping ISI15_0_2.png: mask not found at biomarkem/synthetic_masks_unified/ISI/ISI15_0_2.png
⚠️  Skipping ISI15_0_3.png: mask not found at biomarkem/synthetic_masks_unified/ISI/ISI15_0_3.png
⚠️  Skipping ISI15_0_4.png: mask not found at biomarkem/synthetic_masks_unified/ISI/ISI15_0_4.png
✅ All guidance inpaintings generated.


### Panel to visualize the results

In [44]:
import os
from pathlib import Path
from PIL import Image, ImageDraw, ImageFont
import matplotlib.pyplot as plt
import matplotlib
import numpy as np

def create_guidance_panel(
    image_name,
    scanners=["ISI", "TSI", "VSI"],
    mask_root="./biomarkem/synthetic_masks_unified",
    flair_root="/home/benet/data/biomarkem2D",
    gen_root="./biomarkem/generated_inpaintings_unified_100",
    guidance_values=[0.0, 1.0, 2.0, 3.0, 5.0, 7.5, 10.0],
    output_folder="./biomarkem/guidance_panels_unified_100",
    image_size=512
):
    output_folder = Path(output_folder)
    output_folder.mkdir(parents=True, exist_ok=True)

    # Extract suffix: e.g. ISI01_0_1.png → 01_0_1.png
    suffix = image_name[3:]

    flairs, masks = [], []
    for scanner in scanners:
        fname = f"{scanner}{suffix}"
        flair_path = Path(flair_root) / scanner / fname
        mask_path = Path(mask_root) / scanner / fname

        if not flair_path.exists() or not mask_path.exists():
            print(f"⚠️ Skipping {suffix}: missing FLAIR or MASK for {scanner}")
            return None

        flairs.append(Image.open(flair_path).convert("RGB").resize((image_size, image_size)))
        masks.append(Image.open(mask_path).convert("RGB").resize((image_size, image_size)))

    guidance_rows = []
    for g in guidance_values:
        row = []
        for scanner in scanners:
            fname = f"{scanner}{suffix}"
            gen_path = Path(gen_root) / scanner / fname.replace(".png", "") / f"g{g}.png"
            if not gen_path.exists():
                print(f"⚠️ Skipping {suffix}: missing g={g} for {scanner}")
                return None
            img = Image.open(gen_path).convert("RGB").resize((image_size, image_size))
            row.append(img)
        guidance_rows.append(row)

    rows = [flairs, masks] + guidance_rows

    panel_height = len(rows) * image_size
    panel_width = len(scanners) * image_size
    panel = Image.new("RGB", (panel_width, panel_height), "white")

    for i, row in enumerate(rows):
        for j, img in enumerate(row):
            panel.paste(img, (j * image_size, i * image_size))

    out_path = Path(output_folder) / suffix
    panel.save(out_path)

    return out_path


In [45]:
def batch_create_guidance_panels(image_names):
    for name in image_names:
        try:
            print(f"Generating panel for {name}")
            result = create_guidance_panel(name)
            if result is None:
                print(f"⚠️ Skipped panel for {name} (missing data)")
        except Exception as e:
            print(f"❌ Error with {name}: {e}")

image_names = sorted(os.listdir("/home/benet/data/biomarkem2D/ISI"))  # or from any scanner
batch_create_guidance_panels(image_names)


Generating panel for ISI01_0_0.png
⚠️ Skipping 01_0_0.png: missing FLAIR or MASK for ISI
⚠️ Skipped panel for ISI01_0_0.png (missing data)
Generating panel for ISI01_0_1.png
Generating panel for ISI01_0_2.png
⚠️ Skipping 01_0_2.png: missing FLAIR or MASK for ISI
⚠️ Skipped panel for ISI01_0_2.png (missing data)
Generating panel for ISI01_0_3.png
⚠️ Skipping 01_0_3.png: missing FLAIR or MASK for ISI
⚠️ Skipped panel for ISI01_0_3.png (missing data)
Generating panel for ISI01_0_4.png
⚠️ Skipping 01_0_4.png: missing FLAIR or MASK for ISI
⚠️ Skipped panel for ISI01_0_4.png (missing data)
Generating panel for ISI02_0_0.png
⚠️ Skipping 02_0_0.png: missing FLAIR or MASK for ISI
⚠️ Skipped panel for ISI02_0_0.png (missing data)
Generating panel for ISI02_0_1.png
⚠️ Skipping 02_0_1.png: missing FLAIR or MASK for ISI
⚠️ Skipped panel for ISI02_0_1.png (missing data)
Generating panel for ISI02_0_2.png
Generating panel for ISI02_0_3.png
⚠️ Skipping 02_0_3.png: missing FLAIR or MASK for ISI
⚠️ Skip

In [46]:
def create_guidance_panel(
    image_name,
    scanners=["ISI", "TSI", "VSI"],
    mask_root="./biomarkem/synthetic_masks_unified",
    flair_root="/home/benet/data/biomarkem2D",
    gen_root="./biomarkem/generated_inpaintings_unified",
    guidance_values=[0.0, 1.0, 2.0, 3.0, 5.0, 7.5, 10.0],
    output_folder="./biomarkem/guidance_panels_unified_titles",
    image_size=512,
    font_path="/usr/share/fonts/truetype/dejavu/DejaVuSans-Bold.ttf"
):
    from PIL import ImageDraw, ImageFont

    output_folder = Path(output_folder)
    output_folder.mkdir(parents=True, exist_ok=True)

    suffix = image_name[3:]

    flairs, masks = [], []
    for scanner in scanners:
        fname = f"{scanner}{suffix}"
        flair_path = Path(flair_root) / scanner / fname
        mask_path = Path(mask_root) / scanner / fname
        if not flair_path.exists() or not mask_path.exists():
            print(f"⚠️ Skipping {suffix}: missing FLAIR or MASK for {scanner}")
            return None
        flairs.append(Image.open(flair_path).convert("RGB").resize((image_size, image_size)))
        masks.append(Image.open(mask_path).convert("RGB").resize((image_size, image_size)))

    guidance_rows = []
    for g in guidance_values:
        row = []
        for scanner in scanners:
            fname = f"{scanner}{suffix}"
            gen_path = Path(gen_root) / scanner / fname.replace(".png", "") / f"g{g}.png"
            if not gen_path.exists():
                print(f"⚠️ Skipping {suffix}: missing g={g} for {scanner}")
                return None
            img = Image.open(gen_path).convert("RGB").resize((image_size, image_size))
            row.append(img)
        guidance_rows.append(row)

    row_titles = ["FLAIR", "MASK"] + [f"g={g}" for g in guidance_values]
    col_titles = scanners

    try:
        font = ImageFont.truetype(font_path, 24)
    except:
        font = ImageFont.load_default()

    # Full panel includes space for row and column titles
    panel_width = (len(scanners) + 1) * image_size
    panel_height = (len(row_titles) + 1) * image_size
    panel = Image.new("RGB", (panel_width, panel_height), "white")
    draw = ImageDraw.Draw(panel)

    # Draw column titles
    for j, title in enumerate(col_titles):
        draw.text(((j + 1) * image_size + 10, 10), title, fill="black", font=font)

    # Draw row titles and paste images
    all_rows = [flairs, masks] + guidance_rows
    for i, (row_title, row_images) in enumerate(zip(row_titles, all_rows)):
        draw.text((10, (i + 1) * image_size + 10), row_title, fill="black", font=font)
        for j, img in enumerate(row_images):
            panel.paste(img, ((j + 1) * image_size, (i + 1) * image_size))

    out_path = Path(output_folder) / suffix
    panel.save(out_path)
    return out_path


In [47]:
image_names = sorted(os.listdir("/home/benet/data/biomarkem2D/ISI"))
batch_create_guidance_panels(image_names)

Generating panel for ISI01_0_0.png
⚠️ Skipping 01_0_0.png: missing FLAIR or MASK for ISI
⚠️ Skipped panel for ISI01_0_0.png (missing data)
Generating panel for ISI01_0_1.png
Generating panel for ISI01_0_2.png
⚠️ Skipping 01_0_2.png: missing FLAIR or MASK for ISI
⚠️ Skipped panel for ISI01_0_2.png (missing data)
Generating panel for ISI01_0_3.png
⚠️ Skipping 01_0_3.png: missing FLAIR or MASK for ISI
⚠️ Skipped panel for ISI01_0_3.png (missing data)
Generating panel for ISI01_0_4.png
⚠️ Skipping 01_0_4.png: missing FLAIR or MASK for ISI
⚠️ Skipped panel for ISI01_0_4.png (missing data)
Generating panel for ISI02_0_0.png
⚠️ Skipping 02_0_0.png: missing FLAIR or MASK for ISI
⚠️ Skipped panel for ISI02_0_0.png (missing data)
Generating panel for ISI02_0_1.png
⚠️ Skipping 02_0_1.png: missing FLAIR or MASK for ISI
⚠️ Skipped panel for ISI02_0_1.png (missing data)
Generating panel for ISI02_0_2.png
Generating panel for ISI02_0_3.png
⚠️ Skipping 02_0_3.png: missing FLAIR or MASK for ISI
⚠️ Skip